<a href="https://colab.research.google.com/github/achett/clinical_trial_simulation/blob/main/Trial_Cost_Sim_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
##############
# INSTALL PACKAGES
##############
!pip install simpy

In [9]:
##############
# IMPORT PACKAGES
##############
import simpy
import random
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta

In [10]:
##############
# DATA LOAD
##############
file_path = '/content/sim_inputs v2.csv'
inputs = pd.read_csv(file_path)

file_path = '/content/accrual_period.csv'
accrual_period = pd.read_csv(file_path)

In [11]:
##############
# VARIABLE LOAD
##############

sim_start_date = datetime(2012, 1, 1) # Date of earliest active trial start
data_end_date = datetime(2023, 10, 1) # Date at which actuals end, this is used for adding uncertainty

In [12]:
##############
# SAMPLE
##############
inputs = inputs[inputs['ISN'].isin(trials2sim)]

In [13]:
##############
# TAG
##############
# Concatenate 'CAB' and 'Activity Type' to create 'Cost Account'
accrual_period['Cost Account'] = accrual_period['CAB'] + accrual_period['Activity Type']
inputs['Cost Account'] = inputs['CAB'] + inputs['Activity Type']

<ipython-input-13-e2e82c9d9c29>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs['Cost Account'] = inputs['CAB'] + inputs['Activity Type']


In [14]:
# Function to get a random number of months - replace with your distribution
def get_random_months():
    return np.random.randint(1, 6)  # Randomly choose between 1 to 5 months

def month_diff(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def create_trial_durations_df(inputs, sim_start_date, data_end_date, month_diff_function, get_random_months_function):
    """
    Create a trial durations DataFrame.

    :param inputs: DataFrame containing initial data.
    :param sim_start_date: The simulation start date for time calculations.
    :param data_end_date: The end date for filtering.
    :param month_diff_function: Function to calculate the month difference.
    :param get_random_months_function: Function to generate a random number of months.
    :return: DataFrame with trial durations and updated dates.
    """
    # Convert the date columns to datetime
    inputs['Start Date'] = pd.to_datetime(inputs['Start Date'])
    inputs['End Date'] = pd.to_datetime(inputs['End Date'])

    # Calculating the month differences and creating a new DataFrame
    q1 = inputs.copy()[['ISN', 'Start Spread', 'Start Date']].drop_duplicates()
    q1.columns = ['ISN', 'milestone', 'ds']
    q2 = inputs.copy()[['ISN', 'End Spread', 'End Date']].drop_duplicates()
    q2.columns = ['ISN', 'milestone', 'ds']
    trial_durations = pd.concat([q1, q2]).drop_duplicates()

    # Calculate time2milestone
    trial_durations['time2milestone_orig'] = trial_durations['ds'].apply(lambda x: month_diff(x, sim_start_date))

    # Sort dataframe
    trial_durations = trial_durations.sort_values(by=['ISN', 'ds'], ascending=[True, True]).reset_index(drop=True)

    # Track the total months added for each ISN
    total_months_added = {isn: 0 for isn in trial_durations['ISN'].unique()}

    # Iterate over the DataFrame rows
    for index, row in trial_durations.iterrows():
        if row['ds'] > data_end_date:
            # Draw random months and add to total for this ISN
            additional_months = get_random_months()
            total_months_added[row['ISN']] += additional_months

            # Update the date
            trial_durations.at[index, 'ds'] = row['ds'] + pd.DateOffset(months=total_months_added[row['ISN']])

    # Adjust subsequent dates
    for index, row in trial_durations.iterrows():
        if row['ds'] > data_end_date:
            trial_durations.at[index, 'ds'] += pd.DateOffset(months=total_months_added[row['ISN']])

    # Calculate new time2milestone
    trial_durations['time2milestone'] = trial_durations['ds'].apply(lambda x: month_diff(x, sim_start_date))

    return trial_durations

In [15]:
def create_eac_df(inputs, trial_durations, month_diff_function):
  """
  Create an EAC DataFrame.

  :param inputs: DataFrame containing initial data.
  :param trial_durations: DataFrame with trial duration information.
  :param month_diff_function: Function to calculate the month difference.
  :return: EAC DataFrame.
  """
  # Create EAC df
  eac = inputs.copy()[['Cost Account', 'ISN', 'CAB', 'Activity Type', 'Start Spread', 'End Spread', 'TotalEAC']].drop_duplicates()

  # Add trial milestone dates
  eac = eac.merge(trial_durations[['ISN', 'milestone', 'ds']], how='left', left_on=['ISN', 'Start Spread'], right_on=['ISN', 'milestone'])
  eac = eac.merge(trial_durations[['ISN', 'milestone', 'ds']], how='left', left_on=['ISN', 'End Spread'], right_on=['ISN', 'milestone']).drop_duplicates()

  # Rename columns
  eac.rename(columns={'ds_x': 'Start Date', 'ds_y': 'End Date'}, inplace=True)

  # Find months to spread the costs
  eac['duration2spread'] = eac.apply(lambda row: month_diff(row['End Date'], row['Start Date']), axis=1)

  # Find monthly costs
  eac['monthly_costs'] = eac['TotalEAC'] / eac['duration2spread']

  # Subset
  eac = eac[['Cost Account', 'ISN', 'CAB', 'Activity Type', 'Start Spread', 'End Spread', 'TotalEAC', 'Start Date', 'End Date', 'duration2spread', 'monthly_costs']]

  return eac

In [16]:
##############
# SIMULATION PREP
##############
# Function to check if a cost type is active based on the current time
def is_cost_active(cost_type, current_time, milestone_schedule, accrual_period):
    start_milestone = accrual_period[accrual_period['Cost Account']==cost_type]['Start Milestone'].values[0]
    end_milestone = accrual_period[accrual_period['Cost Account']==cost_type]['End Milestone'].values[0]
    start_time = milestone_schedule[milestone_schedule['milestone']==start_milestone]['time2milestone'].values[0]
    end_time = milestone_schedule[milestone_schedule['milestone']==end_milestone]['time2milestone'].values[0]
    return start_time <= current_time < end_time

# Process for a clinical trial
def clinical_trial(env, trial_name, milestone_schedule, monthly_costs, results):
    # Filter for trial
    milestone_schedule = milestone_schedule[milestone_schedule['ISN']==trial_name]
    monthly_costs = monthly_costs[monthly_costs['ISN']==trial_name]

    # Cost accounst list
    ca_list = accrual_period['Cost Account'].tolist()

    # Initialize costs df
    costs = {cost_type: 0 for cost_type in ca_list}

    current_time = 0  # Start from the beginning of the trials

    while current_time <= milestone_schedule['time2milestone'].max():
        # Update costs for each month
        for cost_type in ca_list:
            if is_cost_active(cost_type, current_time, milestone_schedule, accrual_period):
                costs[cost_type] += monthly_costs[monthly_costs['Cost Account']==cost_type]['monthly_costs'].values[0]

        # Store the costs for this month
        results.append({**{'trial_name': trial_name, 'month': current_time}, **costs})

        # Move to the next month
        yield env.timeout(1)  # Wait for one month
        current_time += 1

In [17]:
##############
# RUN MONTE CARLO SIMULATION
##############
sims=2

def run_monte_carlo(n):
  results = []  # List to collect results

  # Run a for-loop for n_trajectories samples with dummy variable t
  for i in range(n):
    print(i)

    # Update trial milestone uncertainty
    trial_durations = create_trial_durations_df(inputs, sim_start_date, data_end_date, month_diff, get_random_months)

    # Update eac uncertainty
    eac = create_eac_df(inputs, trial_durations, month_diff)

    # Setup and run the simulation
    env = simpy.Environment()
    env.process(clinical_trial(env, trials2sim[0], trial_durations, eac, results))
    env.process(clinical_trial(env, trials2sim[1], trial_durations, eac, results))
    env.run()

    # Print or return the collected results
    print(len(results))
  return results

mc_results=run_monte_carlo(sims)

<ipython-input-14-30e12c5b0764>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs['Start Date'] = pd.to_datetime(inputs['Start Date'])
<ipython-input-14-30e12c5b0764>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs['End Date'] = pd.to_datetime(inputs['End Date'])
<ipython-input-14-30e12c5b0764>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

0
384
1
768


In [18]:
##############
# PROCESS OUTPUT
##############
# Convert the collected data to a DataFrame
mc_results = pd.DataFrame(mc_results)

# List of columns to keep as they are (identifiers) and to melt into 'cost_account'
id_vars = ['trial_name', 'month']
value_vars = mc_results.columns.difference(id_vars)

# Melting the DataFrame
mc_results = mc_results.melt(id_vars=id_vars, value_vars=value_vars, var_name='cost_account', value_name='value')

mc_results

,trial_name,month,cost_account,value
0,2215-CL-0201,0,CTM ServicesSTUDY,0.00
1,8951-CL-0302,0,CTM ServicesSTUDY,0.00
2,2215-CL-0201,1,CTM ServicesSTUDY,0.00
3,8951-CL-0302,1,CTM ServicesSTUDY,0.00
4,2215-CL-0201,2,CTM ServicesSTUDY,0.00
...,...,...,...,...
22267,8951-CL-0302,200,ePROST_SU,738220.31
22268,8951-CL-0302,201,ePROST_SU,738220.31
22269,8951-CL-0302,202,ePROST_SU,738220.31
22270,8951-CL-0302,203,ePROST_SU,738220.31


In [19]:
##############
# VALIDATION
##############
# Group by 'cost_account' and find the max 'value' for each group
max_values = mc_results[mc_results['trial_name']=='2215-CL-0201'].groupby('cost_account')['value'].max()

# Sum up these maximum values
total_sum = max_values.sum()

print("Sum of maximum values by cost account:", total_sum) # 43,430,976

Sum of maximum values by cost account: 43430975.74000003
